## 小批量梯度下降法（MNGB）
它结合了随机梯度下降（SGD）和批量梯度（BGD）下降的概念。 它只是将训练数据集拆分成小批量，并为每个批次执行更新。 因此，它在随机梯度下降的稳健性和批量梯度下降的效率之间创造了平衡。
包括随机抽样和遍历两种方法，在本笔记中使用随机抽样的方法计算。

### 根据$Y=\beta_{0}+\beta_{1}X+\varepsilon $生成线性回归数据

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl


In [2]:
def Beta(X,b):#X为X向量，b为β向量
    X = np.vstack((1,X.reshape(len(X),1)))
    Y = np.dot(b.reshape(1,len(X)),X)
    return Y[0][0]

In [3]:
def random_reg(beta_range,x_num,n,x_range,seed,var_epsilon = 1):#生成线性回归数据
    np.random.seed(seed)
    beta = np.random.uniform(beta_range[0],beta_range[1],x_num+1)
    np.random.seed(seed)
    x = np.random.uniform(x_range[0],x_range[1],x_num*n)
    x = x.reshape(x_num, n)
    Y = []
    for i in range(n):
        X = np.array([x[0:,i]]).reshape(x_num, 1)
        Y.append(Beta(X, beta))
    np.random.seed(seed)
    epsilon = np.random.normal(0, var_epsilon, n)
    Y = Y + epsilon
    return {"beta":beta, "x":x, "Y":Y}

In [5]:
x = random_reg([-5,5], 4, 3200, [0,100], 313,0.01)['x']
beta = random_reg([-5,5], 4, 3200, [0,100], 313,0.01)['beta']
Y = random_reg([-5,5], 4, 3200, [0,100], 313,0.01)['Y']

### 对数据进行分组，并每次随机抽取一组求导函数并进行迭代。

In [6]:
def partical(x, beta_hat, Y):#βhat为估计的β向量
    sum0 = []
    k = len(x[0:,0])
    n = len(x[0, 0:])
    for i in range(n):
        sum0.append(Beta(x[0:,i],beta_hat) - Y[i])
    vector0 = sum(sum0)
    vector = []
    for j in range(k):
        vector.append(sum(np.asarray(sum0) * x[j,0:]))
    vector = 2/n * np.hstack((vector0,vector))
    return vector

In [7]:
def mngb(x, Y, mini_batch, origin, accuracy, alpha,times_num = 1000000):
    col = np.random.randint(len(x[0, 0:]), size = mini_batch)
    x0 = x[0: , col]
    Y0 = Y[col]
    theta = origin - alpha * partical(x0, origin, Y0)
    delta = accuracy +1
    times = 0
    while delta >= accuracy:
        times += 1
        col = np.random.randint(len(x[0, 0:]), size = mini_batch)
        x0 = x[0: , col]
        Y0 = Y[col]
        theta = theta - alpha * partical(x0, theta, Y0)
        delta = alpha * abs(max(partical(x0, theta, Y0)))
        if times >= times_num: break
    return theta

In [8]:
mngb(x, Y, 4, np.array([0, 0, 0, 0, 0]), 1e-6, 1e-5, 1000000)

array([0.0218847 , 0.4984558 , 3.59308105, 1.18748774, 4.41042047])

In [9]:
beta

array([-3.34460376,  0.50104366,  3.60871802,  1.17933724,  4.46246387])